The main solving big ridge regression problems

In [1]:
workspace()

In [2]:
using Plots

In [3]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

plot_outputs_Plots (generic function with 1 method)

In [4]:
#Basic parameters
## Basic parameters
maxiter=10^8;
max_time = 10;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [5]:
dataset = "a9a"
prob = defineProb(dataset);

../data/a9a.jld


In [6]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror) 
OUTPUTS = [];

In [7]:
# Testing Hadamard Sketch
method_name = "Hadamard";# ROS
options.sketchsize =prob.n; # Int(floor(log2(prob.n)));
options.maxiter=100;
options.skip_error_calculation =1;
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

flopsperiter: 1936758
Hadamard
-------------------
It   | Error% | Residual |  Time   
-------------------
  1  | 103851724132.50  |  94868439744.15  | 0.8368 


In [8]:
# Testing CG 
options.max_time = 2; options.restol = 10.0^(-8); 
method_name = "CG";# gradient descent
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
  1  | 98.19  |  21.24  | 0.0927 
  2  | 199.87  |  91.29  | 0.0946 
  3  | 114.64  |  62.60  | 0.0971 
  4  | 195.34  |  61.42  | 0.0976 
  5  | 155.66  |  33.97  | 0.0977 
  6  | 146.09  |  66.15  | 0.0979 
  7  | 152.00  |  241.13  | 0.0981 
  8  | 415.15  |  993.72  | 0.0983 
  9  | 1873.75  |  4296.92  | 0.0985 
 10  | 8651.45  |  19433.91  | 0.0987 
 11  | 41262.25  |  92195.18  | 0.0989 
 12  | 206352.51  |  460502.88  | 0.0992 


In [9]:
# Testing grad descent
options.max_time = 2; options.restol = 10.0^(-8); options.maxiter=1000;
method_name = "grad";# gradient descent
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

grad
-------------------
It   | Error% | Residual |  Time   
-------------------
  1  | 98.19  |  21.24  | 0.2311 
  2  | 182.77  |  42.95  | 0.2432 
  3  | 192.34  |  100.23  | 0.2536 
  4  | 161.38  |  46.82  | 0.2584 
  5  | 137.54  |  68.24  | 0.2606 
  6  | 212.19  |  87.80  | 0.2617 
  7  | 214.90  |  320.00  | 0.2625 
  8  | 2782.62  |  1269.63  | 0.2626 
  9  | 3069.36  |  4746.49  | 0.2626 
 10  | 32304.67  |  14900.09  | 0.2627 
 11  | 36088.27  |  55994.43  | 0.2628 
 12  | 354789.93  |  164380.33  | 0.2629 


In [10]:
# Testing CD 
options.max_time = 10; options.restol = 10.0^(-8); options.maxiter=10000;
method_name = "CD";# gradient descent
options.sketchsize = 11#floor((prob.n)^(1/2));
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD
-------------------
It   | Error% | Residual |  Time   
-------------------
  1  | 162.13  |  54.83  | 0.2075 
  2  | 172.58  |  19.68  | 0.2076 
  3  | 176.02  |  11.07  | 0.2077 
  4  | 179.90  |  9.55  | 0.2077 
  5  | 182.19  |  8.43  | 0.2077 
  6  | 181.27  |  7.71  | 0.2078 
  7  | 180.13  |  7.50  | 0.2078 
  8  | 180.13  |  7.19  | 0.2079 
  9  | 179.98  |  7.10  | 0.2079 
 10  | 180.37  |  6.67  | 0.2079 
 11  | 182.27  |  6.44  | 0.2079 
 12  | 185.90  |  5.90  | 0.2080 
 13  | 186.41  |  6.04  | 0.2080 
 14  | 185.99  |  5.62  | 0.2080 
 15  | 182.41  |  5.78  | 0.2081 
 16  | 169.87  |  8.63  | 0.2081 
 17  | 170.08  |  4.48  | 0.2081 
 18  | 169.82  |  4.41  | 0.2081 
 19  | 169.35  |  3.65  | 0.2082 
 20  | 169.03  |  4.23  | 0.2082 
 21  | 166.70  |  4.99  | 0.2082 
 22  | 165.48  |  2.35  | 0.2083 
 23  | 161.12  |  2.00  | 0.2083 
 24  | 161.21  |  2.02  | 0.2083 
 25  | 161.68  |  1.61  | 0.2083 
 26  | 162.15  |  1.41  | 0.2084 
 27  | 164.88  |  1.21  | 0.2084 


264  | 161.16  |  0.08  | 0.2147 
265  | 161.17  |  0.06  | 0.2148 
266  | 161.09  |  0.06  | 0.2148 
267  | 161.26  |  0.06  | 0.2148 
268  | 161.27  |  0.06  | 0.2149 
269  | 161.26  |  0.07  | 0.2149 
270  | 161.26  |  0.06  | 0.2149 
271  | 161.26  |  0.06  | 0.2150 
272  | 161.26  |  0.07  | 0.2150 
273  | 161.25  |  0.07  | 0.2150 
274  | 161.25  |  0.07  | 0.2150 
275  | 161.25  |  0.07  | 0.2151 
276  | 161.25  |  0.07  | 0.2151 
277  | 161.26  |  0.07  | 0.2151 
278  | 161.28  |  0.07  | 0.2151 
279  | 161.26  |  0.07  | 0.2152 
280  | 161.25  |  0.07  | 0.2152 
281  | 161.22  |  0.07  | 0.2152 
282  | 161.22  |  0.07  | 0.2152 
283  | 161.23  |  0.07  | 0.2153 
284  | 161.22  |  0.07  | 0.2153 
285  | 161.22  |  0.07  | 0.2153 
286  | 161.22  |  0.07  | 0.2153 
287  | 161.22  |  0.07  | 0.2154 
288  | 161.22  |  0.07  | 0.2154 
289  | 161.22  |  0.07  | 0.2154 
290  | 161.22  |  0.07  | 0.2154 
291  | 161.22  |  0.07  | 0.2155 
292  | 161.18  |  0.07  | 0.2155 
293  | 161.19 

594  | 160.12  |  0.01  | 0.2234 
595  | 160.14  |  0.01  | 0.2234 
596  | 160.14  |  0.01  | 0.2234 
597  | 160.16  |  0.01  | 0.2235 
598  | 160.16  |  0.01  | 0.2235 
599  | 160.16  |  0.01  | 0.2236 
600  | 160.18  |  0.01  | 0.2236 
601  | 160.18  |  0.01  | 0.2236 
602  | 160.19  |  0.01  | 0.2237 
603  | 160.17  |  0.01  | 0.2237 
604  | 160.16  |  0.01  | 0.2237 
605  | 160.16  |  0.01  | 0.2237 
606  | 160.16  |  0.01  | 0.2238 
607  | 160.18  |  0.01  | 0.2238 
608  | 160.18  |  0.01  | 0.2238 
609  | 160.18  |  0.02  | 0.2238 
610  | 160.23  |  0.02  | 0.2239 
611  | 160.23  |  0.01  | 0.2239 
612  | 160.22  |  0.01  | 0.2239 
613  | 160.24  |  0.01  | 0.2240 
614  | 160.24  |  0.01  | 0.2240 
615  | 160.24  |  0.01  | 0.2240 
616  | 160.24  |  0.01  | 0.2240 
617  | 160.24  |  0.01  | 0.2241 
618  | 160.24  |  0.01  | 0.2242 
619  | 160.25  |  0.01  | 0.2242 
620  | 160.25  |  0.02  | 0.2242 
621  | 160.25  |  0.02  | 0.2243 
622  | 160.25  |  0.01  | 0.2243 
623  | 160.25 

905  | 160.64  |  0.00  | 0.2371 
906  | 160.64  |  0.00  | 0.2372 
907  | 160.64  |  0.00  | 0.2372 
908  | 160.63  |  0.00  | 0.2372 
909  | 160.63  |  0.00  | 0.2373 
910  | 160.63  |  0.00  | 0.2373 
911  | 160.63  |  0.00  | 0.2373 
912  | 160.63  |  0.00  | 0.2373 
913  | 160.63  |  0.00  | 0.2374 
914  | 160.63  |  0.00  | 0.2374 
915  | 160.64  |  0.00  | 0.2374 
916  | 160.64  |  0.00  | 0.2374 
917  | 160.64  |  0.00  | 0.2374 
918  | 160.64  |  0.00  | 0.2375 
919  | 160.64  |  0.00  | 0.2375 
920  | 160.64  |  0.00  | 0.2375 
921  | 160.64  |  0.00  | 0.2375 
922  | 160.64  |  0.00  | 0.2376 
923  | 160.64  |  0.00  | 0.2376 
924  | 160.64  |  0.00  | 0.2376 
925  | 160.64  |  0.00  | 0.2376 
926  | 160.64  |  0.00  | 0.2377 
927  | 160.64  |  0.00  | 0.2377 
928  | 160.63  |  0.00  | 0.2377 
929  | 160.63  |  0.00  | 0.2377 
930  | 160.63  |  0.00  | 0.2378 
931  | 160.63  |  0.00  | 0.2378 
932  | 160.63  |  0.00  | 0.2378 
933  | 160.63  |  0.00  | 0.2378 
934  | 160.63 

1237  | 160.70  |  0.00  | 0.2449 
1238  | 160.70  |  0.00  | 0.2449 
1239  | 160.71  |  0.00  | 0.2450 
1240  | 160.70  |  0.00  | 0.2450 
1241  | 160.70  |  0.00  | 0.2450 
1242  | 160.71  |  0.00  | 0.2450 
1243  | 160.71  |  0.00  | 0.2451 
1244  | 160.71  |  0.00  | 0.2451 
1245  | 160.71  |  0.00  | 0.2451 
1246  | 160.71  |  0.00  | 0.2451 
1247  | 160.71  |  0.00  | 0.2451 
1248  | 160.71  |  0.00  | 0.2452 
1249  | 160.71  |  0.00  | 0.2452 
1250  | 160.71  |  0.00  | 0.2452 
1251  | 160.71  |  0.00  | 0.2452 
1252  | 160.71  |  0.00  | 0.2453 
1253  | 160.71  |  0.00  | 0.2453 
1254  | 160.71  |  0.00  | 0.2453 
1255  | 160.71  |  0.00  | 0.2453 
1256  | 160.71  |  0.00  | 0.2454 
1257  | 160.71  |  0.00  | 0.2454 
1258  | 160.71  |  0.00  | 0.2454 
1259  | 160.71  |  0.00  | 0.2454 
1260  | 160.71  |  0.00  | 0.2454 
1261  | 160.71  |  0.00  | 0.2455 
1262  | 160.71  |  0.00  | 0.2455 
1263  | 160.71  |  0.00  | 0.2455 
1264  | 160.71  |  0.00  | 0.2455 
1265  | 160.71  |  0

...

2511  | 160.71  |  0.00  | 0.2824 
2512  | 160.71  |  0.00  | 0.2824 
2513  | 160.71  |  0.00  | 0.2824 
2514  | 160.71  |  0.00  | 0.2825 
2515  | 160.71  |  0.00  | 0.2825 
2516  | 160.71  |  0.00  | 0.2825 
2517  | 160.71  |  0.00  | 0.2826 
2518  | 160.71  |  0.00  | 0.2826 
2519  | 160.71  |  0.00  | 0.2826 
2520  | 160.71  |  0.00  | 0.2826 
2521  | 160.71  |  0.00  | 0.2827 
2522  | 160.71  |  0.00  | 0.2827 
2523  | 160.71  |  0.00  | 0.2827 
2524  | 160.71  |  0.00  | 0.2827 
2525  | 160.71  |  0.00  | 0.2827 
2526  | 160.71  |  0.00  | 0.2828 
2527  | 160.71  |  0.00  | 0.2828 
2528  | 160.71  |  0.00  | 0.2828 
2529  | 160.71  |  0.00  | 0.2828 
2530  | 160.71  |  0.00  | 0.2829 
2531  | 160.71  |  0.00  | 0.2829 
2532  | 160.71  |  0.00  | 0.2831 
2533  | 160.71  |  0.00  | 0.2831 
2534  | 160.71  |  0.00  | 0.2832 
2535  | 160.71  |  0.00  | 0.2832 
2536  | 160.71  |  0.00  | 0.2832 
2537  | 160.71  |  0.00  | 0.2833 
2538  | 160.71  |  0.00  | 0.2833 
2539  | 160.71  |  0

2824  | 160.71  |  0.00  | 0.2900 
2825  | 160.71  |  0.00  | 0.2900 
2826  | 160.71  |  0.00  | 0.2900 
2827  | 160.71  |  0.00  | 0.2901 
2828  | 160.71  |  0.00  | 0.2901 
2829  | 160.71  |  0.00  | 0.2901 
2830  | 160.71  |  0.00  | 0.2902 
2831  | 160.71  |  0.00  | 0.2902 
2832  | 160.71  |  0.00  | 0.2902 
2833  | 160.71  |  0.00  | 0.2902 
2834  | 160.71  |  0.00  | 0.2903 
2835  | 160.71  |  0.00  | 0.2903 
2836  | 160.71  |  0.00  | 0.2904 
2837  | 160.71  |  0.00  | 0.2904 
2838  | 160.71  |  0.00  | 0.2904 
2839  | 160.71  |  0.00  | 0.2904 
2840  | 160.71  |  0.00  | 0.2904 
2841  | 160.71  |  0.00  | 0.2905 
2842  | 160.71  |  0.00  | 0.2905 
2843  | 160.71  |  0.00  | 0.2905 
2844  | 160.71  |  0.00  | 0.2905 
2845  | 160.71  |  0.00  | 0.2906 
2846  | 160.71  |  0.00  | 0.2906 
2847  | 160.71  |  0.00  | 0.2906 
2848  | 160.71  |  0.00  | 0.2906 
2849  | 160.71  |  0.00  | 0.2906 
2850  | 160.71  |  0.00  | 0.2907 
2851  | 160.71  |  0.00  | 0.2907 
2852  | 160.71  |  0

3061  | 160.71  |  0.00  | 0.2956 
3062  | 160.71  |  0.00  | 0.2957 
3063  | 160.71  |  0.00  | 0.2957 
3064  | 160.71  |  0.00  | 0.2957 
3065  | 160.71  |  0.00  | 0.2958 
3066  | 160.71  |  0.00  | 0.2958 
3067  | 160.71  |  0.00  | 0.2958 
3068  | 160.71  |  0.00  | 0.2958 
3069  | 160.71  |  0.00  | 0.2959 
3070  | 160.71  |  0.00  | 0.2959 
3071  | 160.71  |  0.00  | 0.2959 
3072  | 160.71  |  0.00  | 0.2959 
3073  | 160.71  |  0.00  | 0.2960 
3074  | 160.71  |  0.00  | 0.2960 
3075  | 160.71  |  0.00  | 0.2960 
3076  | 160.71  |  0.00  | 0.2961 
3077  | 160.71  |  0.00  | 0.2961 
3078  | 160.71  |  0.00  | 0.2961 
3079  | 160.71  |  0.00  | 0.2961 
3080  | 160.71  |  0.00  | 0.2962 
3081  | 160.71  |  0.00  | 0.2962 
3082  | 160.71  |  0.00  | 0.2962 
3083  | 160.71  |  0.00  | 0.2962 
3084  | 160.71  |  0.00  | 0.2962 
3085  | 160.71  |  0.00  | 0.2963 
3086  | 160.71  |  0.00  | 0.2963 
3087  | 160.71  |  0.00  | 0.2963 
3088  | 160.71  |  0.00  | 0.2963 
3089  | 160.71  |  0

3360  | 160.71  |  0.00  | 0.3098 
3361  | 160.71  |  0.00  | 0.3099 
3362  | 160.71  |  0.00  | 0.3099 
3363  | 160.71  |  0.00  | 0.3099 
3364  | 160.71  |  0.00  | 0.3100 
3365  | 160.71  |  0.00  | 0.3100 
3366  | 160.71  |  0.00  | 0.3100 
3367  | 160.71  |  0.00  | 0.3100 
3368  | 160.71  |  0.00  | 0.3101 
3369  | 160.71  |  0.00  | 0.3101 
3370  | 160.71  |  0.00  | 0.3101 
3371  | 160.71  |  0.00  | 0.3102 
3372  | 160.71  |  0.00  | 0.3102 
3373  | 160.71  |  0.00  | 0.3102 
3374  | 160.71  |  0.00  | 0.3102 
3375  | 160.71  |  0.00  | 0.3103 
3376  | 160.71  |  0.00  | 0.3103 
3377  | 160.71  |  0.00  | 0.3103 
3378  | 160.71  |  0.00  | 0.3103 
3379  | 160.71  |  0.00  | 0.3104 
3380  | 160.71  |  0.00  | 0.3104 
3381  | 160.71  |  0.00  | 0.3104 
3382  | 160.71  |  0.00  | 0.3105 
3383  | 160.71  |  0.00  | 0.3105 
3384  | 160.71  |  0.00  | 0.3105 
3385  | 160.71  |  0.00  | 0.3105 
3386  | 160.71  |  0.00  | 0.3106 
3387  | 160.71  |  0.00  | 0.3106 
3388  | 160.71  |  0

3651  | 160.71  |  0.00  | 0.3176 
3652  | 160.71  |  0.00  | 0.3176 
3653  | 160.71  |  0.00  | 0.3177 
3654  | 160.71  |  0.00  | 0.3177 
3655  | 160.71  |  0.00  | 0.3177 
3656  | 160.71  |  0.00  | 0.3177 
3657  | 160.71  |  0.00  | 0.3178 
3658  | 160.71  |  0.00  | 0.3178 
3659  | 160.71  |  0.00  | 0.3178 
3660  | 160.71  |  0.00  | 0.3178 
3661  | 160.71  |  0.00  | 0.3179 
3662  | 160.71  |  0.00  | 0.3179 
3663  | 160.71  |  0.00  | 0.3179 
3664  | 160.71  |  0.00  | 0.3179 
3665  | 160.71  |  0.00  | 0.3180 
3666  | 160.71  |  0.00  | 0.3180 
3667  | 160.71  |  0.00  | 0.3180 
3668  | 160.71  |  0.00  | 0.3180 
3669  | 160.71  |  0.00  | 0.3180 
3670  | 160.71  |  0.00  | 0.3181 
3671  | 160.71  |  0.00  | 0.3181 
3672  | 160.71  |  0.00  | 0.3181 
3673  | 160.71  |  0.00  | 0.3181 
3674  | 160.71  |  0.00  | 0.3182 
3675  | 160.71  |  0.00  | 0.3182 
3676  | 160.71  |  0.00  | 0.3182 
3677  | 160.71  |  0.00  | 0.3182 
3678  | 160.71  |  0.00  | 0.3182 
3679  | 160.71  |  0

In [ ]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [ ]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

In [ ]:
# Loading saved problem data
probname = "w8a"#"gisette_scale-ridge";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];
# mask = [1,3,5]
# OUTPUTS[mask]